In [1]:
# Importing all the necessary libraries
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import numpy as np
import calendar
import re

In [2]:
# Read the CSV file into a DataFrame
df = pd.read_csv('2024_MPesa_Statements.csv')

# Get column names as a list
column_names = df.columns.tolist()

# Print the list of column names to see if everything is okay with the columns
print(column_names)

['Receipt_No', 'Completion_Time', 'Unnamed: 2', 'Details', 'Transaction_Status', 'Received', 'Withdrawn', 'Unnamed: 7', 'Balance', 'Unnamed: 9']


In [3]:
# As we can see, we have unnecessary columns that we need to drp
# Dropping 'Unnamed: 2', 'Unnamed: 7', and 'Unnamed: 9'
df = df.loc[:, ~df.columns.str.contains('Unnamed')]

In [4]:
# Now lets check the data types of the data in
print(df.dtypes)

Receipt_No            object
Completion_Time       object
Details               object
Transaction_Status    object
Received              object
Withdrawn             object
Balance               object
dtype: object


In [5]:
# We will need to convert these data types into the respective data items they represent

In [6]:
# Converting the Received, Withdrawn and Balance Columns to float data type
df['Received'] = df['Received'].astype(str).str.replace(',', '').astype(float)
df['Withdrawn'] = df['Withdrawn'].astype(str).str.replace(',', '').astype(float).abs() # The 'Withdrawn' column has negative values so  I have used the absolute function to make them positive
df['Balance'] = df['Balance'].astype(str).str.replace(',', '').astype(float)

In [7]:
# Converting the 'Completion_Time' to datetime format and split into 'Date' and 'Time' so that I can work with it better
df['Completion_Time'] = pd.to_datetime(df['Completion_Time'], errors='coerce')
df['Date'] = df['Completion_Time'].dt.date
df['Time'] = df['Completion_Time'].dt.time

# Removing the old 'Completion_Time' column since we now have new 'Date' and 'Time' columns
df.drop(columns=['Completion_Time'], inplace=True)

In [8]:
# Now let's check the data types again to see if they have been updated
print(df.dtypes)

Receipt_No             object
Details                object
Transaction_Status     object
Received              float64
Withdrawn             float64
Balance               float64
Date                   object
Time                   object
dtype: object


In [ ]:
# Hopefully we are done with data cleaning. If not we will just deal with them as they come 😂😂😂

In [11]:
# 1. The total amount of money I received in 2024 via M-Pesa

# Calculating the total sum in the 'Received Column'
total_received = df['Received'].sum()

# Printing the result
print(f"I received a total of KES {total_received:,.2f} in 2024 on my M-Pesa account.")

I received a total of KES 965,967.00 in 2024 on my M-Pesa account.


In [12]:
# 2. The total amount of money I sent via M-Pesa in 2024

# Calculating the total sum in the 'Withdrawn Column'
total_spent = df['Withdrawn'].sum()

# Printing the result
print(f"I spent a total of KES {total_spent:,.2f} in 2024 using my M-Pesa account.")

I spent a total of KES 981,420.00 in 2024 using my M-Pesa account.


In [13]:
# 3. My net earnings in year 2024

# The net earnings is the difference between the total money I received and the total money I sent using M-Pesa
my_earnings = total_received - total_spent

# Printing the result
print(f"I earned KES {my_earnings:,.2f} in 2024 on my M-Pesa account.")

I earned KES -15,453.00 in 2024 on my M-Pesa account.


In [14]:
# So basically I finished the year with a deficit of KSHS 15,453.00
# Very interesting 😂😂😂
# But in my defense, I had some cash in my M-Shwari account 😒😂😒


# Now let's see the breakdown of the money I spent in this year using Send Money, Pochi la Biashara, Till Numbers and Paybill Numbers

In [15]:
# 4. The amount of money I spent in 2024 using Send Money, Pochi la Biashara, Till Numbers and Paybill Numbers services

# Defining transaction categories based on 'Details' column, identify and sum up amounts spent in each category (Send Money, Pochi la Biashara, Till Numbers and Paybill Numbers services)
send_money_total = df[df['Details'].str.startswith('Customer Transfer', na=False)]['Withdrawn'].sum()
pochi_total = df[df['Details'].str.startswith('Customer Payment to Small Business', na=False)]['Withdrawn'].sum()
till_total = df[df['Details'].str.startswith('Merchant Payment', na=False)]['Withdrawn'].sum()
paybill_total = df[df['Details'].str.startswith('Pay Bill', na=False)]['Withdrawn'].sum()
mshwari_deposit_total = df[df['Details'].str.startswith('M-Shwari Deposit', na=False)]['Withdrawn'].sum()
airtime_total = df[df['Details'].str.startswith('Airtime Purchase', na=False)]['Withdrawn'].sum()

# Storing the total amounts spent per category in the 'categories' dictionary
categories = {
    "Send Money": send_money_total,  # Money sent to individuals (Send Money)
    "Pochi la Biashara": pochi_total,  # Payments to small businesses (Pochi la Biashara)
    "Till Numbers": till_total,  # Payments made to business Till numbers
    "Paybill Numbers": paybill_total,  # Payments made via Paybill numbers
    "M-Shwari Deposit": mshwari_deposit_total,  # Deposits made to M-Shwari
    "Airtime Purchase": airtime_total  # Amount spent on Buying Airtime
}

# Converting the dictionary into a DataFrame for better tabular representation
df_summary = pd.DataFrame(list(categories.items()), columns=["Transaction Type", "Amount (KSHS)"])

# Formating the "Amount (KES)" column with comma separators for readability
df_summary["Amount (KSHS)"] = df_summary["Amount (KSHS)"].apply(lambda x: f"{x:,.2f}")

# Printing the table showing each category and the total amount I spent in each category
print(df_summary.to_string(index=False))

 Transaction Type Amount (KSHS)
       Send Money    202,065.00
Pochi la Biashara     34,997.00
     Till Numbers    116,959.00
  Paybill Numbers    147,628.00
 M-Shwari Deposit    405,296.00
 Airtime Purchase      6,640.00


In [16]:
# As we can see, I spent the most amount of money on M-Shwari Deposits (405,296.00) and the least amount of money on Airtime Purchases (6,640) in 2024
# You remember I had mentioned I had some cash on my M-Shwari😂😂👌😂😂

# Let's visualize these results for better viewing and interpretation
# Note that the graph is interactive and you can hover over the elements of the graph to get more insights

In [17]:
# 4.1 Bar chart of the amount of money I spent on Send Money, Pochi la Biashara, Till Numbers and Paybill Numbers services in 2024

# Converting the 'categories' dictionary to a DataFrame
df_bar = pd.DataFrame(list(categories.items()), columns=["Transaction Type", "Amount (KES)"])

# Calculating percentages
total_spent = df_bar['Amount (KES)'].sum()
df_bar['Percentage'] = (df_bar['Amount (KES)'] / total_spent) * 100

# Formatting percentages as strings with one decimal place
df_bar['Percentage_Text'] = df_bar['Percentage'].apply(lambda x: f"{x:.1f}%")

# Creating the bar chart using Plotly Express
fig = px.bar(
    df_bar,
    x="Transaction Type",
    y="Amount (KES)",
    title="Bar Graph of the amount of money I spent on Send Money, Pochi la Biashara, Till Numbers and Paybill Numbers services in 2024",
    color="Transaction Type",
    color_discrete_sequence=px.colors.qualitative.Pastel,
    text='Percentage_Text'
)

# Customizing the layout for better readability
fig.update_layout(
    title_font_size = 14,
    title_font_color = "blue",
    title_x=0.5,
    xaxis_title="Transaction Type",
    yaxis_title="Amount (KES)",
    yaxis_tickformat = ',.0f'
)

# Showing the bar chart
fig.show()

In [18]:
# 4.2 Results Verification

# We can verify the results by calculating the total amount spent across all categories to see if it matches the amount I had calculated as my total spending (KSHS 981,420.00)
total_categorical_spending = (
    send_money_total +
    pochi_total +
    till_total +
    paybill_total +
    mshwari_deposit_total +
    airtime_total
)
print(f"I spent KES {total_spent:,.2f} across all categories in 2024 using M-Pesa.")

I spent KES 913,585.00 across all categories in 2024 using M-Pesa.


In [19]:
# Viola!!!😂😂😂
# We see that the amount I have calculated above matches my total spending on M-Pesa for 2024😊😊😊
# Furthermore, this result matches the one displayed on my M-Pesa App😂😂😂
# This means the code is correct and we are good to proceed 😂😂😂

In [20]:
# By the way it looks like I was sending a lot of money to my M-Shwari account 😂😂😂
# Don't ask me why I was storing away all my cash 😂😂😂
# On the other hand though, I sent most of my cash to send money accounts
# We will see which person I sent the most amount of money later on 😊😊😊
# A few Paybills, Tills and Pochi's received my money 😂😪😂
# I don't know why I spent so little on Airtime. Perhaps its because I have very few friends😂😂😂
# Anyway, let's go ahead and look deeper into my monthly spending patterns

In [21]:
# 5. My monthly spending in the year 2024

# Extracting Year-Month for grouping
df['Date'] = pd.to_datetime(df['Date']) # Ensuring 'Date' is datetime type
df['Month'] = df['Date'].dt.strftime('%B %Y')

# Calculating my total spending per month
monthly_spendings = df.groupby('Month')['Withdrawn'].sum().reset_index()

# Renaming columns for better readability
monthly_spendings.columns = ['Month', 'Total Spent (KSHS)']

# Sorting the months in chronological order
monthly_spendings['Date_Sort'] = pd.to_datetime(monthly_spendings['Month'], format='%B %Y')
monthly_spendings = monthly_spendings.sort_values(by='Date_Sort').drop(columns=['Date_Sort'])

# Formatting the currency for readability
monthly_spendings['Total Spent (KSHS)'] = monthly_spendings['Total Spent (KSHS)'].apply(lambda x: f"{x:,.2f}")

# Displaying the monthly spendings table
print(monthly_spendings.to_string(index=False))

         Month Total Spent (KSHS)
  January 2024          31,685.00
 February 2024          37,496.00
    March 2024          71,219.00
    April 2024          16,338.00
      May 2024          88,755.00
     June 2024         117,400.00
     July 2024         137,713.00
   August 2024          21,854.00
September 2024          57,634.00
  October 2024          83,762.00
 November 2024         100,695.00
 December 2024         216,869.00


In [22]:
# As we can see, I spent the most amount of money in December(KSHS 216,869.00) and the least amount of money in April (KSHS 16,338).
# April was a tough month for the comrade😂😂😂.
# For December it goes without saying ... it was Christmas period😂😂😎😎👌👌

# Now let's visualize these results for better analysis.
# Note that the graph is interactive and you can hover over the elements of the graph to get more insights

In [23]:
# 5.1 Bar chart of my monthly spending in the year 2024

# Calculating my total spending per month
monthly_spendings = df.groupby('Month')['Withdrawn'].sum().reset_index()

# Renaming the columns for better readability
monthly_spendings.columns = ['Month', 'Total Spent (KSHS)']

# Sorting the months in chronological order
monthly_spendings['Date_Sort'] = pd.to_datetime(monthly_spendings['Month'], format='%B %Y')
monthly_spendings = monthly_spendings.sort_values(by='Date_Sort').drop(columns=['Date_Sort'])

# Formatting the currency for readability
monthly_spendings['Total Spent (KSHS)'] = monthly_spendings['Total Spent (KSHS)'].apply(lambda x: f"{x:,.2f}")

# Converting 'Total Spent (KSHS)' to numeric after removing commas
monthly_spendings['Total Spent (KSHS)'] = monthly_spendings['Total Spent (KSHS)'].str.replace(',', '').astype(float)

# Calculating the total spending for all months
total_spending = monthly_spendings['Total Spent (KSHS)'].sum()

# Calculating the percentage of spending for each month
monthly_spendings['Percentage'] = (monthly_spendings['Total Spent (KSHS)'] / total_spending) * 100

# Formatting the percentages as strings with one decimal place and a '%' sign
monthly_spendings['Percentage_Text'] = monthly_spendings['Percentage'].apply(lambda x: f"{x:.1f}%")


# Creating the bar chart using Plotly Express
fig = px.bar(
    monthly_spendings,
    x="Month",
    y="Total Spent (KSHS)",
    title="Bar chart of my monthly spending in the year 2024",
    labels={"Total Spent (KSHS)": "Total Spent (KSHS)", "Month": "Month"},
    color="Month",
    color_discrete_sequence=px.colors.sequential.Turbo,
    text='Percentage_Text'
)

# Customizing the layout for better readability
fig.update_layout(
    xaxis_tickangle=-45,
    title_font_size = 14,
    title_font_color = "blue",
    title_x=0.5,
    yaxis_tickformat = ',.0f'
)

# Showing the bar chart
fig.show()

In [24]:
# 5.2 Results Verification

# Again, we can verify the results by calculating the total amount spent in all the 12 months to see if it matches the amount I had calculated as my total spending (KSHS 981,420.00)
# Creating a dictionary with all my monthly spendings
monthly_spendings = {
    "January 2024": 31685.00,
    "February 2024": 37496.00,
    "March 2024": 71219.00,
    "April 2024": 16338.00,
    "May 2024": 88755.00,
    "June 2024": 117400.00,
    "July 2024": 137713.00,
    "August 2024": 21854.00,
    "September 2024": 57634.00,
    "October 2024": 83762.00,
    "November 2024": 100695.00,
    "December 2024": 216869.00,
}

# Calculating the total spending for the year
total_spending = sum(monthly_spendings.values())

# Printing the result
print(f"Total spending in 2024: KES {total_spending:,.2f}")

Total spending in 2024: KES 981,420.00


In [25]:
# Yaay😂👌😂
# It matches😂😂😂
# And this result also matches the one displayed on the M-Pesa app😎😂😎

# Now lets dive deeper and look at my daily spendings

In [27]:
# 6. Analysing my daily transactions on M-Pesa
# In this section I will retrieve the day I spent the highest amount of money (excluding depositing to M-Swari because that is not really an expenditure) and details about the transactions
# I will also create a graph of all the transactions for the year 2024

# Ensuring 'Details' column is of string type
df['Details'] = df['Details'].astype(str)

# Filtering out M-Shwari Deposits
df_filtered = df[~df['Details'].str.contains("M-Shwari Deposit", case=False, na=False)]

# Grouping by 'Date' and summing 'Withdrawn' amounts
daily_spendings = df_filtered.groupby('Date', as_index=False)['Withdrawn'].sum()

# Finding the day with the highest spending
max_spent_day = daily_spendings.loc[daily_spendings['Withdrawn'].idxmax()]

# Extracting details of the highest spending day
highest_spending_date = max_spent_day['Date']
highest_spent_amount = max_spent_day['Withdrawn']

# Getting all transactions for that day (excluding M-Shwari deposits)
highest_spending_day_records = df_filtered[df_filtered['Date'] == highest_spending_date].copy()

# Calculating cumulative spending for that day
highest_spending_day_records['Cumulative Amount'] = highest_spending_day_records['Withdrawn'].cumsum()

# Printing the highest spending day summary
print(f"\n💸 I spent the highest amount of money on {highest_spending_date.strftime('%A, %d %B %Y')}, "
      f"with a total spending of KSHS {highest_spent_amount:,.2f} (excluding M-Shwari deposits).\n")

print("📜 **Detailed Transactions for That Day:**\n")
print(highest_spending_day_records[['Date', 'Details', 'Withdrawn', 'Cumulative Amount']].to_string(index=False))


💸 I spent the highest amount of money on Monday, 20 May 2024, with a total spending of KSHS 25,915.00 (excluding M-Shwari deposits).

📜 **Detailed Transactions for That Day:**

      Date                                                                 Details  Withdrawn  Cumulative Amount
2024-05-20           Merchant Payment Online to 7203691 - MAWINGU  AIRTIME LIMITED      199.0              199.0
2024-05-20                   Merchant Payment Online to 913237 - CISBEN   AGENCIES      100.0              299.0
2024-05-20 Customer Payment to Small Business to - 2547******454 Alexander King'oo       80.0              379.0
2024-05-20                                                                     nan       20.0              399.0
2024-05-20   Customer Payment to Small Business to - 2547******740 TENAI KIPCHUMBA       50.0              449.0
2024-05-20                   Customer Transfer to - 2547******128 JOSHUA MUTHANGYA       58.0              507.0
2024-05-20                     

In [28]:
# We can see I sent someone (I won't mention who for security purposes) money that's why the spending for that day was the highest.
# Let's filter out send money options

In [29]:
# 6.1 expenditure without Send Money Option

# Filter out M-Shwari Deposits and Customer Transfers
df_filtered = df[~df['Details'].str.contains("M-Shwari Deposit|Customer Transfer", case=False, na=False)]

# Grouping by 'Date' and summing 'Withdrawn' amounts
daily_spendings = df_filtered.groupby('Date', as_index=False)['Withdrawn'].sum()

# Finding the day with the highest spending
max_spent_day = daily_spendings.loc[daily_spendings['Withdrawn'].idxmax()]

# Extracting details of the highest spending day
highest_spending_date = max_spent_day['Date']
highest_spent_amount = max_spent_day['Withdrawn']

# Getting all transactions for that day (excluding M-Shwari deposits and customer transfers)
highest_spending_day_records = df_filtered[df_filtered['Date'] == highest_spending_date].copy()

# Calculating cumulative spending for that day
highest_spending_day_records['Cumulative Amount'] = highest_spending_day_records['Withdrawn'].cumsum()

# Printing the highest spending day summary
print(f"\n💸 I spent the highest amount of money on {highest_spending_date.strftime('%A, %d %B %Y')}, "
      f"with a total spending of KSHS {highest_spent_amount:,.2f} (excluding M-Shwari deposits and customer transfers).\n")

print("📜 **Detailed Transactions for That Day:**\n")
print(highest_spending_day_records[['Date', 'Details', 'Withdrawn', 'Cumulative Amount']].to_string(index=False))


💸 I spent the highest amount of money on Monday, 09 September 2024, with a total spending of KSHS 23,099.00 (excluding M-Shwari deposits and customer transfers).

📜 **Detailed Transactions for That Day:**

      Date                                                            Details  Withdrawn  Cumulative Amount
2024-09-09             Merchant Payment to 259046 - mathai supermarkets Ruiru      162.0              162.0
2024-09-09 Customer Payment to Small Business to - 2547******555 LUCY NDUNG'U      300.0              462.0
2024-09-09 Customer Payment to Small Business to - 2547******061 MORRIS MAINA      270.0              732.0
2024-09-09                                                    Pay Bill Charge       67.0              799.0
2024-09-09       Pay Bill to 247247 - Equity  Paybill Account Acc. 0766885881    22000.0            22799.0
2024-09-09                                              M-Shwari Loan Request        NaN                NaN
2024-09-09                           

In [30]:
# Now this is more like it. I remember I paid school fees on that day😂😂😂
# That was the day I spent the most amount of money in the year 😂😂😂

In [31]:
# 6.2 Bar chart of my daily spending in 2024
# Note that the graph is interactive and you can hover over the elements of the graph to get more insights

# Grouping by 'Date' and sum the 'Withdrawn' amounts
daily_spendings = df.groupby('Date', as_index=False)['Withdrawn'].sum()

# Calculating total spending for the entire period
total_spending = daily_spendings['Withdrawn'].sum()

# Calculating the percentage of total spending for each day
daily_spendings['Percentage'] = (daily_spendings['Withdrawn'] / total_spending) * 100

# Creating a formatted percentage text for hover
daily_spendings['Percentage_Text'] = daily_spendings['Percentage'].apply(lambda x: f"{x:.1f}%")

# Creating an interactive line chart using Plotly
fig = px.line(
    daily_spendings,
    x='Date',
    y='Withdrawn',
    markers=True,
    title="Bar chart of my daily spending in 2024",
    labels={"Withdrawn": "Total Spent (KES)", "Date": "Date"}
)

# Customizing the hover tooltip to show the percentage
fig.update_traces(
    hovertemplate="Date: %{x}<br>Total Spent (KES): %{y:,.2f}<br>Percentage of Total Spending: %{customdata}",
    customdata=daily_spendings['Percentage_Text'],
    line=dict(width=2, color='blue')
)

# Showing my bar chart of my daily spending in 2024
fig.show()

In [32]:
# Now that we have seen my highest expenditure day and my expenditure graph for the year, lets now focus on how much I spent on transaction cost charges

In [33]:
# 7. My montly total transaction costs charges for 2024 (Send Money, Pochi la Biashara, Till Numbers and Paybill Numbers services)

# Extracting Year, Month, and Year-Month Name
df['Year'] = df['Date'].dt.year
df['Month'] = df['Date'].dt.month
df['Month Name'] = df['Month'].apply(lambda x: calendar.month_name[x])
df['Year-Month'] = df['Month Name'] + " " + df['Year'].astype(str)

# Defining M-PESA transaction costs based on the tariffs on their website
TARIFFS = {
    "Send Money": [
        (1, 49, 0), (50, 100, 0), (101, 500, 7), (501, 1000, 13),
        (1001, 1500, 23), (1501, 2500, 33), (2501, 3500, 53),
        (3501, 5000, 57), (5001, 7500, 78), (7501, 10000, 90),
        (10001, 15000, 100), (15001, 20000, 105), (20001, 35000, 108),
        (35001, 50000, 108), (50001, 250000, 108)
    ],
    "Withdraw M-PESA Agent": [
        (50, 100, 11), (101, 500, 29), (501, 1000, 29),
        (1001, 1500, 29), (1501, 2500, 29), (2501, 3500, 52),
        (3501, 5000, 69), (5001, 7500, 87), (7501, 10000, 115),
        (10001, 15000, 167), (15001, 20000, 185), (20001, 35000, 197),
        (35001, 50000, 278), (50001, 250000, 309)
    ],
    "ATM Withdrawal": [
        (200, 2500, 35), (2501, 5000, 69), (5001, 10000, 115), (10001, 35000, 203)
    ]
}

# Function to determine transaction type
def get_transaction_type(details):
    if pd.isna(details):
        return None
    details = str(details)  # Ensure string format
    if details.startswith("Customer Transfer"):
        return "Send Money"
    elif details.startswith("Customer Payment to Small Business"):
        return "Send Money"
    elif details.startswith("Withdrawal from M-PESA Agent"):
        return "Withdraw M-PESA Agent"
    elif details.startswith("ATM Withdrawal"):
        return "ATM Withdrawal"
    return None

# Function to calculate transaction cost
def calculate_transaction_cost(amount, transaction_type):
    if pd.isna(amount) or transaction_type not in TARIFFS:
        return 0
    for min_val, max_val, charge in TARIFFS[transaction_type]:
        if min_val <= amount <= max_val:
            return charge
    return 0

# Applying transaction type and cost calculations
df['Transaction Type'] = df['Details'].apply(get_transaction_type)
df['Transaction Cost'] = df.apply(lambda row: calculate_transaction_cost(row['Withdrawn'], row['Transaction Type']), axis=1)

# Generating Yearly Transaction Summary
yearly_summary = df.groupby("Year").agg(
    Number_of_Transactions=("Transaction Cost", "count"),
    Transaction_Cost=("Transaction Cost", "sum")
).reset_index()

# Formatting transaction cost
yearly_summary["Transaction Cost (KES)"] = yearly_summary["Transaction_Cost"].apply(lambda x: f"{x:,.2f}")
yearly_summary = yearly_summary.drop(columns=["Transaction_Cost"])

# Generating Monthly Transaction Summary
monthly_summary = df.groupby("Year-Month").agg(
    Number_of_Transactions=("Transaction Cost", "count"),
    Transaction_Cost=("Transaction Cost", "sum")
).reset_index()

# Formatting transaction cost
monthly_summary["Transaction Cost (KES)"] = monthly_summary["Transaction_Cost"].apply(lambda x: f"{x:,.2f}")
monthly_summary = monthly_summary.drop(columns=["Transaction_Cost"])

# Ensuring months are sorted in correct order
monthly_summary['Month Number'] = monthly_summary['Year-Month'].apply(lambda x: list(calendar.month_name).index(x.split()[0]))
monthly_summary = monthly_summary.sort_values(by=['Month Number']).drop(columns=['Month Number'])

# Displaying the results
print("📊 Monthly M-Pesa Transactions and Costs")
print(monthly_summary.to_string(index=False))

print("\n📊 Yearly M-Pesa Transaction Summary")
print(yearly_summary.to_string(index=False))

📊 Monthly M-Pesa Transactions and Costs
    Year-Month  Number_of_Transactions Transaction Cost (KES)
  January 2024                     121                  88.00
 February 2024                     121                 114.00
    March 2024                     217                 491.00
    April 2024                     107                 123.00
      May 2024                     156                 290.00
     June 2024                     215                 512.00
     July 2024                     208                 292.00
   August 2024                     115                  68.00
September 2024                     130                  49.00
  October 2024                     228                  73.00
 November 2024                     265                 358.00
 December 2024                     212                 378.00

📊 Yearly M-Pesa Transaction Summary
 Year  Number_of_Transactions Transaction Cost (KES)
 2024                    2095               2,836.00


In [34]:
# From the above information we can get my average number of transactions and the average transaction costs in a day

In [35]:
# 7.1 My average number of transactions and the average transaction costs in a day

# Grouping by 'Date' and counting the number of transactions for each day
daily_transactions = df.groupby('Date')['Transaction Cost'].count().reset_index()

# Calculating the average number of transactions per day
average_transactions_per_day = daily_transactions['Transaction Cost'].mean()

# Display the result
print(f"My average number of transactions per day in 2024 is: {average_transactions_per_day:.2f}")

# Grouping by 'Date' and summing the transaction costs for each day
daily_transaction_costs = df.groupby('Date')['Transaction Cost'].sum().reset_index()

# Calculating the average transaction cost per day
average_transaction_cost_per_day = daily_transaction_costs['Transaction Cost'].mean()

# Display the result
print(f"My average transaction cost per day in 2024 is: KSHS {average_transaction_cost_per_day:,.2f}")

My average number of transactions per day in 2024 is: 6.14
My average transaction cost per day in 2024 is: KSHS 8.32


In [36]:
# Let me visualize the total monthly transaction costs
# # Note that the graph is interactive and you can hover over the elements of the graph to get more insights

In [37]:
# 7.2 Bar chart for my total monthly transaction cost charges in 2024

# Creating the bar chart using Plotly Express
fig = px.bar(
    monthly_summary,
    x="Year-Month",
    y="Transaction Cost (KES)",
    title="Bar chart for my total monthly transaction cost charges in 2024",
    labels={"Transaction Cost (KES)": "Transaction Cost (KES)", "Year-Month": "Month"},
    color="Year-Month",
    color_discrete_sequence=px.colors.sequential.Turbo,
    text='Transaction Cost (KES)'
)

# Customizing the layout for better readability
fig.update_layout(
    xaxis_tickangle=-45,
    title_font_size = 14,
    title_font_color = "blue",
    title_x=0.5,
    yaxis_tickformat = ',.0f'
)

# Showing the bar chart
fig.show()

In [39]:
# 8. The largest amount of cash I received in 2024 including everything

# Filtering transactions from 2024
df_2024 = df[df['Date'].dt.year == 2024]

# Removing NaN values in 'Received' column
df_2024 = df_2024.dropna(subset=['Received'])

# Finding the largest amount received and the sender
max_received_row = df_2024.loc[df_2024['Received'].idxmax()]
max_received_amount = max_received_row['Received']
max_sender = max_received_row['Details']

# Finding the lowest amount received and the sender
min_received_row = df_2024.loc[df_2024['Received'].idxmin()]
min_received_amount = min_received_row['Received']
min_sender = min_received_row['Details']

# Displaying the results
print("📊 Received Amount Summary for 2024")
print(f"💰 Largest Amount Received: KES {max_received_amount:,.2f} from {max_sender}")
print(f"🪙 Lowest Amount Received: KES {min_received_amount:,.2f} from {min_sender}")

📊 Received Amount Summary for 2024
💰 Largest Amount Received: KES 24,000.00 from M-Shwari Withdraw
🪙 Lowest Amount Received: KES 0.00 from M-Shwari Loan Request


In [40]:
# Since I was constantly moving money to and from my account🤦‍♂️🤦‍♂️🤦‍♂️, we can safely ignore the M-Shwari transactions😂😂😂

In [41]:
# 8.1 The largest amount of money I received in 2024 excluding M-Shwari transactions

# Filtering transactions from 2024, excluding M-Shwari transactions
df_2024 = df[(df['Date'].dt.year == 2024) & (~df['Details'].str.contains("M-Shwari", na=False))]

# Removing NaN values in 'Received' column
df_2024 = df_2024.dropna(subset=['Received'])

# Finding the largest amount received and the sender
max_received_row = df_2024.loc[df_2024['Received'].idxmax()]
max_received_amount = max_received_row['Received']
max_sender = max_received_row['Details']

# Finding the lowest amount received and the sender
min_received_row = df_2024.loc[df_2024['Received'].idxmin()]
min_received_amount = min_received_row['Received']
min_sender = min_received_row['Details']

# Displaying the results
print("📊 Received Amount Summary for 2024 (Excluding M-Shwari)")
print(f"💰 Largest Amount Received: KES {max_received_amount:,.2f} from {max_sender}")
print(f"🪙 Lowest Amount Received: KES {min_received_amount:,.2f} from {min_sender}")


📊 Received Amount Summary for 2024 (Excluding M-Shwari)
💰 Largest Amount Received: KES 22,000.00 from Business Payment from 329299 - STANDARD CHARTERED BANK
via API. Original conversation ID is 254746152008-KE-IBNK-
4uyeyM.
🪙 Lowest Amount Received: KES 5.00 from Funds received from - 07******478 esther wanjohi


In [42]:
# The largest amount received was sent so that I pay for my school fees😂😂😂
# The lowest amount ... I dont even know what to say😂😂😂
# I guess someone wanted to know my name so they sent me 5 bobo so that M-Pesa displays my name😂😂😂

In [43]:
# 9. Where I spent most of my money on

# Filtering transactions from 2024 with non-null Withdrawn values
df_2024_spent = df[(df['Date'].dt.year == 2024) & df['Withdrawn'].notna()]

# Grouping by 'Details' and sum up the 'Withdrawn' amounts
spending_summary = df_2024_spent.groupby('Details')['Withdrawn'].sum().reset_index()

# Finding the detail with the highest cumulative spend
max_spent_row = spending_summary.loc[spending_summary['Withdrawn'].idxmax()]
max_spent_detail = max_spent_row['Details']
max_spent_amount = max_spent_row['Withdrawn']

# Displaying results
print(f"💸 I spent the most of money in 2024 on: {max_spent_detail} (KSHS {max_spent_amount:,.2f})")

💸 I spent the most of money in 2024 on: M-Shwari Deposit (KSHS 405,296.00)


In [44]:
# I really wonder why i was moving money aroud my M-Shwari this much and this often 🤦‍♂️🤦‍♂️🤦‍♂️

In [47]:
# 9.1 Where I spent most of my money on excluding M-Shwari transactions
# Filter transactions from 2024, excluding M-Shwari, with non-null Withdrawn values
df_2024_spent = df[
    (df['Date'].dt.year == 2024) &
    df['Withdrawn'].notna() &
    (~df['Details'].str.contains("M-Shwari", na=False))
]

# Group by 'Details' and sum up the 'Withdrawn' amounts
spending_summary = df_2024_spent.groupby('Details')['Withdrawn'].sum().reset_index()

# Find the detail with the highest cumulative spend
max_spent_row = spending_summary.loc[spending_summary['Withdrawn'].idxmax()]
max_spent_detail = max_spent_row['Details']
max_spent_amount = max_spent_row['Withdrawn']

# Display results
print(f"💸 I spent the most on: {max_spent_detail} (KES {max_spent_amount:,.2f})")

💸 I spent the most on: Pay Bill Online to 247247 - Equity Paybill Account Acc. 0880183523934 (KES 48,900.00)


In [46]:
# This is my school's (Jomo Kenyatta Univeristy of Agriculture and Technology) school fees payment account 😂😂😂
# So it goes without saying that I paid school fees twice to this account in 2024 and that is why its popping up here😂😂😂

# I know you are tired of me at this point😂😂😂
# So let's wrap it up with an analysis of how many times I bought Mutura (African Sausage for those of you who aren't kenyan) in 2024 and for how much😂😂😂

In [48]:
# Escape the special characters in the search string
search_string = "Customer Payment to Small Business to - 2547******543 GIDRAPH GACIBU"

# Filter transactions for GIDRAPH GACIBU in 2024
df_gidraph = df[
    (df['Date'].dt.year == 2024) &
    (df['Details'].str.contains(search_string, na=False, regex=False)) &
    df['Withdrawn'].notna()
]

# Count the number of transactions
num_transactions = len(df_gidraph)

# Group data by month, summing the withdrawn amount and counting the transactions
monthly_gidraph = df_gidraph.groupby(df_gidraph['Date'].dt.month).agg(
    Withdrawn=('Withdrawn', 'sum'),
    Transactions=('Withdrawn', 'count')
).reset_index()

# Add month names
monthly_gidraph['Month Name'] = monthly_gidraph['Date'].apply(lambda x: calendar.month_name[x])

# Total amount spent in the year
total_amount_2024 = df_gidraph['Withdrawn'].sum()

# Display results
print("📊 My Mutura Purchases in 2024")
print(f"💸 I bought mutura from my plug {num_transactions} times in 2024.")

print("\n📅 Monthly Breakdown:")
print(monthly_gidraph[['Month Name', 'Transactions', 'Withdrawn']].to_string(index=False))

print(f"\n🔹 This means I spent a total of KES {total_amount_2024:,.2f} on mutura in 2024 😂😂😂")

📊 My Mutura Purchases in 2024
💸 I bought mutura from my plug 62 times in 2024.

📅 Monthly Breakdown:
Month Name  Transactions  Withdrawn
   January             7      350.0
  February             8      400.0
     March            11      550.0
     April             4      200.0
      June             2      100.0
 September             5      250.0
   October            11      620.0
  November             8      550.0
  December             6      350.0

🔹 This means I spent a total of KES 3,370.00 on mutura in 2024 😂😂😂


In [49]:
# Creating the bar chart for transactions and withdrawn amounts
fig = px.bar(
    monthly_gidraph,
    x="Month Name",
    y=["Transactions", "Withdrawn"],
    title="Monthly Breakdown of my Mutura Purchases in 2024",
    labels={"Transactions": "Number of Transactions", "Withdrawn": "Amount Withdrawn (KES)"},
    barmode="group"  # To display the bars next to each other for comparison
)

# Customizing the layout for better readability
fig.update_layout(
    xaxis_tickangle=-45,
    title_font_size=16,
    title_font_color="blue",
    title_x=0.5,
    yaxis_tickformat=",.0f"  # Formatting y-axis for currency
)

# Showing the bar chart
fig.show()

In [50]:
# I am done😂😂😂
# Thankyou for sticking around